In [1]:
import cv2, socket, numpy, pickle, os 
s=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 10000000)
serverip="0.0.0.1"
serverport=111

In [4]:
import torch
import torchvision
import cv2
import argparse
import time
import numpy as np

In [5]:
names = "/nvdli-nano/data/action_recognition_kinetics.txt"

In [8]:
parser = argparse.ArgumentParser()

parser.add_argument('-f', '--file', required=False)
parser.add_argument('-l', '--clip-len', dest='clip_len', default=16, type=int,
                    help='number of frames to consider for each prediction')
parser.add_argument('-c', '--classes', default = names, help='Path to classes list.')

args = vars(parser.parse_args())
#### PRINT INFO #####
print(f"Number of frames to consider for each prediction: {args['clip_len']}")

Number of frames to consider for each prediction: 16


In [9]:
# get the lables
class_names = open(args["classes"]).read().strip().split("\n")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device is {device}")

# load the model
model = torchvision.models.video.r2plus1d_18(pretrained=True, progress=True) #This one works.

# load the model onto the computation device
model = model.eval().to(device)

Device is cuda


Downloading: "https://download.pytorch.org/models/r2plus1d_18-91a641e6.pth" to /root/.cache/torch/hub/checkpoints/r2plus1d_18-91a641e6.pth
100.0%


In [10]:
from jetcam.csi_camera import CSICamera
fps_origin = 30
#camera = CSICamera(width=224, height=224)

camera = CSICamera(width=400, height=225, capture_width=1280, capture_height=720, capture_fps=fps_origin)

In [12]:
frame_count = 0 # to count total frames
total_fps = 0 # to get the final frames per second
# a clips list to append and store the individual frames
clips = []
frame_and_metadatas = []

In [16]:
import torchvision.transforms as T
# define the transforms
# This cell may need to be ran twice, ignore the first run error.
transform = T.Compose([
    T.Resize((128, 171)),
    T.CenterCrop((112, 112)),
    T.Normalize(mean = [0.43216, 0.394666, 0.37645],
                std = [0.22803, 0.22145, 0.216989], )
])

In [ ]:
# read until end of video
from google.colab.patches import cv2_imshow
processed=0
while(cap.isOpened()):
    # capture each frame of the video
    frame = camera.read()
    # get the start time
    start_time = time.time()
    image = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = transform(image=frame)['image']
    clips.append(frame)
    if len(clips) == args['clip_len']:
        with torch.no_grad(): # we do not want to backprop any gradients
            input_frames = np.array(clips)
            # add an extra dimension        
            input_frames = np.expand_dims(input_frames, axis=0)
            # transpose to get [1, 3, num_clips, height, width]
            input_frames = np.transpose(input_frames, (0, 4, 1, 2, 3))
            # convert the frames to tensor
            input_frames = torch.tensor(input_frames, dtype=torch.float32)
            input_frames = input_frames.to(device)
            # forward pass to get the predictions
            outputs = model(input_frames)
            # get the prediction index
            _, preds = torch.max(outputs.data, 1)

            # map predictions to the respective class names
            label = class_names[preds].strip()
        # get the end time
        end_time = time.time()
        # get the fps
        fps = 1 / (end_time - start_time)
        # add fps to total fps
        total_fps += fps
        # increment frame count
        frame_count += 1
        wait_time = max(1, int(fps/4))
        label_with_time_stamp = label + f"  frame #: {frame_count}, time: {(cap.get(cv2.CAP_PROP_POS_MSEC))/1000:.2f}s"
        cv2.putText(image, label_with_time_stamp, (15, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2, 
                    lineType=cv2.LINE_AA)
        clips.pop(0)
        if frame_count % 20 == 0:
        #     cv2_imshow(image) # colab이 아닌 그냥 Ipynb에서는 cv.imshow() 쓰면 돼요
          ret, buffer = cv2.imencode(".jpg", image, [int(cv2.IMWRITE_JPEG_QUALITY), 30])    
          x_as_bytes = pickle.dumps(buffer)    

          s.sendto(x_as_bytes,(serverip , serverport))
        # out.write(image)

        # if label.strip() not in category2group:
        #     group = "others"
        # else:
        #     group = category2group[label.strip()]
        # meta_dict = {"label":label, "pos":frame_count, "group":group}
        # frame_and_metadatas.append(meta_dict)

        # press `q` to exit
        if cv2.waitKey(wait_time) & 0xFF == ord('q'):
            break